In [ ]:
# 22/01/09
# 건우
# yolov4 결과 모델을 tflite 파일로 변경

In [4]:
# save_model.py 와 convert.py가 있는 폴더로 이동


%cd /content/drive/MyDrive/tflite/

/content/drive/MyDrive/tflite


In [11]:
# https://github.com/theAIGuysCode/tensorflow-yolov4-tflite 다운

!git clone 'https://github.com/theAIGuysCode/tensorflow-yolov4-tflite' '/content/drive/MyDrive/tflite/git/'


Cloning into '/content/drive/MyDrive/tflite/git'...
remote: Enumerating objects: 912, done.
remote: Total 912 (delta 0), reused 0 (delta 0), pack-reused 912
Receiving objects: 100% (912/912), 211.46 MiB | 14.34 MiB/s, done.
Resolving deltas: 100% (441/441), done.
Checking out files: 100% (133/133), done.


In [5]:
# save_model.py가 있는곳으로 이동

%cd /content/drive/MyDrive/tflite/git/


/content/drive/MyDrive/tflite/git


In [2]:
# Save tf model for tflite converting

!python save_model.py --weights ../data/yolov4_custom_last.weights --output ../checkpoints/yolov4-416 --input_size 416 --model yolov4 --framework tflite

2022-01-09 06:09:42.834832: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Traceback (most recent call last):
  File "save_model.py", line 58, in <module>
    app.run(main)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "save_model.py", line 54, in main
    save_tf()
  File "save_model.py", line 49, in save_tf
    utils.load_weights(model, FLAGS.weights, FLAGS.model, FLAGS.tiny)
  File "/content/drive/My Drive/tflite/git/core/utils.py", line 63, in load_weights
    conv_weights = conv_weights.reshape(conv_shape).transpose([2, 3, 1, 0])
ValueError: cannot reshape array of size 4710717 into shape (1024,512,3,3)


In [4]:
# 에러해결방법
# https://github.com/hunglc007/tensorflow-yolov4-tflite/issues/126
# class가 안맞아서 data/coco.names에서 클래스 다시설정후 실행

!python save_model.py --weights ../data/yolov4_custom_last.weights --output ../checkpoints/yolov4-416 --input_size 416 --model yolov4 --framework tflite


2022-01-09 06:41:09.666819: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 416, 416, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                          

In [7]:
# yolov4 quantize float16

!python convert_tflite.py --weights ../checkpoints/yolov4-416 --output ../checkpoints/yolov4-416-fp16.tflite --quantize_mode float16

2022-01-09 07:06:17.636784: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-01-09 07:06:35.534839: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-01-09 07:06:35.534901: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-01-09 07:06:35.534918: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
loc("batch_normalization/moving_mean"): error: is not immutable, try removing mutable variables in your model since mutable variables are currently not supported through this converter
Traceback (most recent call last):
  File "convert_tflite.py", line 76, in <module>
    app.run(main)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, ar

In [4]:
# 위의 오류를 해결하기위해 tensorflow 버전 변경
# https://stackoverflow.com/questions/65407156/errors-in-converting-tensorflow-model-to-tensorflow-lite-model

import tensorflow as tf

if tf.__version__ != '2.3.0-rc0':
  !pip uninstall -y tensorflow
  !pip install tensorflow-gpu==2.3.1

     |████████████████████████████████| 320.4 MB 26 kB/s 
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.3.0
    Uninstalling tensorflow-gpu-2.3.0:
      Successfully uninstalled tensorflow-gpu-2.3.0


In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.1


In [6]:
# yolov4 quantize float16

!python convert_tflite.py --weights ../checkpoints/yolov4-416 --output ../checkpoints/yolov4-416-fp16.tflite --quantize_mode float16

2022-01-09 07:29:03.809866: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-01-09 07:29:05.719789: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-09 07:29:05.743304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 07:29:05.744110: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2022-01-09 07:29:05.744156: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-01-09 07:29:05.746557: I tensorflow/stream_executor/platform/defa